![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2FDev%2Fnew&dt=Python+Asynchronous+API+Calls.ipynb)

# Python Asynchronous API Calls

Method for making asynchronous API calls.  Additionally, managing concurrent request and handling errors.

---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Getting%20Started%20-%20Vertex%20AI%20GenAI%20Python%20Client.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [4]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [5]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [6]:
REGION = 'us-central1'
SERIES = 'tips'
EXPERIMENT = 'async-api'

packages:

In [46]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import vertexai.language_models
import vertexai.preview.generative_models

clients:

In [8]:
vertexai.init(project = PROJECT_ID, location = REGION)

In [39]:
palm_model = vertexai.language_models.TextGenerationModel.from_pretrained("text-bison-32k@002")

In [40]:
vocab_words = palm_model.predict(
    prompt = "I need 600+ unique vocabulary words to study for the GMAT.  Respond with only a comma separate list of word like: word, word, ...",
    max_output_tokens = 8000
)

In [41]:
vocab_words = [word.strip() for word in vocab_words.text.split(',')]

In [42]:
vocab_words[0:10] + [f'... ({len(vocab_words) - 20} more words)'] + vocab_words[-10:]

['abdicate',
 'aberrant',
 'abet',
 'abeyance',
 'abhor',
 'abject',
 'abjure',
 'abrogate',
 'abscond',
 'abstemious',
 '... (657 more words)',
 'wince',
 'whimsical',
 'wily',
 'wince',
 'wrangle',
 'wrath',
 'zealous',
 'zenith',
 'zephyr',
 'zestful']

In [47]:
gemini_model = vertexai.preview.generative_models.GenerativeModel("gemini-pro")

In [52]:
gemini_model.generate_content(['Define the word', vocab_words[0]]).text

"Abdicate means to formally renounce one's throne, high office, or position of authority. It involves voluntarily giving up power or responsibility that one holds. Abdication often occurs when a monarch or other leader decides to step down from their position, usually due to old age, health reasons, or personal choice. The act of abdication is often accompanied by a formal ceremony or announcement, where the individual publicly declares their decision to relinquish their title or office."

In [56]:
palm_model.predict(prompt = f'Define the word {vocab_words[0]}', max_output_tokens = 500).text

' Abdicate means to give up or resign from a position of power or responsibility, especially a throne or high office. It is often used in the context of a monarch or other head of state voluntarily relinquishing their position, but can also be used more broadly to refer to any situation where someone gives up a position of authority or responsibility.\n\nFor example, a king or queen might abdicate the throne in favor of their heir, or a CEO might abdicate their position in favor of a successor. In a more general sense, someone might abdicate their responsibilities as a parent or guardian, or a student might abdicate their responsibilities as a class president.\n\nThe word abdicate comes from the Latin word abdicare, which means "to renounce" or "to give up." It has been used in English since the 14th century.'

In [58]:
await palm_model.predict_async(prompt = f'Define the word {vocab_words[0]}', max_output_tokens = 500)

 Abdicate means to give up or resign from a position of power or responsibility, especially a throne or high office. It is often used in the context of a monarch or other head of state voluntarily relinquishing their position, but can also be used more broadly to refer to any situation where someone gives up a position of authority or responsibility.

For example, a king or queen might abdicate the throne in favor of their heir, or a CEO might abdicate their position in favor of a successor. In a more general sense, someone might abdicate their responsibilities as a parent or guardian, or a student might abdicate their responsibilities as a class president.

The word abdicate comes from the Latin word abdicare, which means "to renounce" or "to give up." It has been used in English since the 14th century.